In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Load environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

# Eassy Writer
    Plan -> Research_Plan -> Write -?-> Critique -> Research_Critique
    Research_Critique -> Write -?-> END

In [2]:
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

# Create a fresh in-memory database
shared_memory = SqliteSaver(sqlite3.connect(":memory:", check_same_thread=False))

In [4]:
from pydantic import BaseModel
from typing import List
from tavily import TavilyClient

class Queries(BaseModel):
    queries: List[str]

tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

In [5]:
from typing import TypedDict, List

from langchain_openai import ChatOpenAI


class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int


model = ChatOpenAI(
    model="qwen/qwen3-4b-2507",
    base_url="http://localhost:1234/v1",
    temperature=0.0,
    api_key=OPENAI_API_KEY,
)

In [26]:
PLAN_PROMPT = """
You are a expert writer tasked with writing a high level outline of an essay. \
Write such a outline for the user provided topic. Give an outline of the essay along with any relevant notes \
or instructions for the sections.
""".strip()

RESEARCH_PLAN_PROMPT = """
You are a researcher charged with providing information that can \
be used when writing the following essay. Generate a list of search queries that will gather \
any relevant information. Only generate 3 queries max.""".strip()


WRITER_PROMPT = """
You are an essay assistant tasked with writing excellent 5-paragraph essays.\
Generate the best essay possible for the user's request and the initial outline. \
If the user provides critique, respond with a revised version of your previous attempts. \
Utilize all the information below as needed: 

------

{content}""".strip()

CRITIQUE_PROMPT = """
You are a teacher grading an essay submission. \
Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests \
    for length, depth, style, etc.""".strip()

RESEARCH_CRITIQUE_PROMPT = """
You are a researcher charged with providing information that can \
be used when making any requested revisions (as outlined below). \
Generate a list of search queries that will gather any relevant information. \
    Only generate 3 queries max.""".strip()


from typing import Dict, Any
from langchain_core.messages import SystemMessage, HumanMessage

def plan_agent(state: AgentState) -> Dict[str, Any]:
    messages = [
        SystemMessage(content=PLAN_PROMPT),
        HumanMessage(content=state["task"]),
    ]
    response = model.invoke(messages)
    return {"plan": response.content}

def research_plan_agent(state: AgentState) -> Dict[str, Any]:
    messages = [
        SystemMessage(content=RESEARCH_PLAN_PROMPT),
        HumanMessage(content=state["task"]),
    ]

    content = ('content' in state and state["content"]) or []
    queries = model.with_structured_output(Queries).invoke(messages)
    for q in queries.queries:
        response = tavily_client.search(q, max_results=2)
        for result in response['results']:
            content.append(result['content'])

    return {"content": content}

def writer_agent(state: AgentState) -> Dict[str, Any]:
    content = '\n\n'.join(state['content'])
    messages = [
        SystemMessage(content=WRITER_PROMPT.format(content=content)),
        HumanMessage(content=f'{state["task"]}\n\nHere is my outline:\n\n{state["plan"]}'),
    ]
    response = model.invoke(messages)
    return {
        "draft": response.content,
        "revision_number": state.get("revision_number", 1) + 1,
    }

def critique_agent(state: AgentState) -> Dict[str, Any]:
    messages = [
        SystemMessage(content=CRITIQUE_PROMPT),
        HumanMessage(content=state["draft"]),
    ]
    response = model.invoke(messages)
    return {"critique": response.content}

def research_critique_agent(state: AgentState) -> Dict[str, Any]:
    messages = [
        SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
        HumanMessage(content=state["critique"]),
    ]
    queries = model.with_structured_output(Queries).invoke(messages)
    content = ('content' in state and state["content"]) or []
    for q in queries.queries:
        response = tavily_client.search(q, max_results=2)
        for result in response['results']:
            content.append(result['content'])

    return {"content": content}

def should_stop_agent(state: AgentState) -> Dict[str, Any]:
    return state.get("revision_number", 1) >= state.get("max_revisions", 5)


In [28]:
from langgraph.graph import StateGraph, END

graph_builder = StateGraph(AgentState)
graph_builder.add_node("plan", plan_agent)
graph_builder.add_node("research_plan", research_plan_agent)
graph_builder.add_node("writer", writer_agent)
graph_builder.add_node("critique", critique_agent)
graph_builder.add_node("research_critique", research_critique_agent)

graph_builder.add_edge("plan", "research_plan")
graph_builder.add_edge("research_plan", "writer")
graph_builder.add_edge("writer", "critique")
graph_builder.add_edge("critique", "research_critique")
graph_builder.add_edge("research_critique", "writer")
graph_builder.add_conditional_edges("writer", should_stop_agent, {True: END, False: "critique"})


graph_builder.set_entry_point("plan")
graph = graph_builder.compile(checkpointer=shared_memory)

In [29]:
thread = {'configurable': {'thread_id': '1'}}
message = {
    'task': 'what is the difference between a cat and a dog?',
    'max_revisions': 2,
    "revision_number": 1
}

for event in graph.stream(message, thread):
    print(event)

{'plan': {'plan': '**Essay Outline: The Difference Between a Cat and a Dog**\n\n---\n\n**I. Introduction**  \n*Purpose:* Introduce the central topic and establish the significance of understanding the differences between cats and dogs.  \n- Briefly define both animals as common household pets.  \n- State the purpose: to explore and compare the behavioral, social, physical, and emotional differences between cats and dogs.  \n- Thesis statement: While both cats and dogs are beloved companions, they differ significantly in temperament, social needs, care requirements, and human interaction—shaping how they fit into different lifestyles and family dynamics.\n\n---\n\n**II. Behavioral and Temperament Differences**  \n*Purpose:* Highlight how cats and dogs approach human interaction and daily routines.  \n- **Cats:**  \n  - Independent, self-sufficient, and often more aloof.  \n  - Prefer solitude and may not seek constant attention.  \n  - Exhibit territorial behavior and may be more reserv

BadRequestError: Error code: 400 - {'error': 'The number of tokens to keep from the initial prompt is greater than the context length. Try to load the model with a larger context length, or provide a shorter input'}